In [1]:
import json 
import requests
import re

from collections import Counter
from spacy.lang.en.stop_words import STOP_WORDS

In [2]:
import pandas as pd 

import seaborn as sns 
import spacy
import matplotlib.pyplot as plt

In [3]:
import nltk

In [4]:
import plotly
plotly.tools.set_credentials_file(username='japprcnj2', api_key='sQkQA7w4dT5EpCdcz0ZG')

In [5]:
import plotly.plotly as py 

In [6]:
import plotly.graph_objs as go

Each item will be represented by a dictionary which we can then easily load into a df where the keys are the column names. id, text, date, tag, creator, title 

In [9]:
item_list = [] # a list of the dictionary objects 

In [10]:
with open("cache_texts.json") as json_file:
    data = json.load(json_file)
    for page in data: 
        for item in data[page]:
            new_item_dict = {"id" : item["id"], "type": [], "tags": []}
            for x in item["element_texts"]:
                if x["element"]["name"] == "Text":
                    new_item_dict["text"] = x["text"]
                if x["element"]["name"] == "Date":
                    new_item_dict["date"] = x["text"]
                if x["element"]["name"] == "Creator":
                    new_item_dict["creator"] = x["text"]
                if x["element"]["name"] == "Title":
                    new_item_dict["title"] = x["text"]
                if x["element"]["name"] == "Type":
                    new_item_dict["type"].append(x["text"])
            for x in item["tags"]:
                new_item_dict["tags"].append(x["name"])
            item_list.append(new_item_dict)

In [11]:
item_df = pd.DataFrame(item_list).dropna()
item_df.head()

,creator,date,id,tags,text,title,type
0,"Addams, Jane",1907-09-05,17,"[Industry, Labor, Writings]","<div style=""text-align: center;"">EFFECT OF FAM...","Effect of Familiarity, September 5, 1907",[article]
1,"Addams, Jane",1906-02-01,18,"[Education, Immigrants, Lectures]","<p style=""text-align: center;"">ELECTIVE OR APP...","Elective or Appointive School Board, February ...",[speech]
2,"Addams, Jane",1914-01-31,19,"[Travels, Writings]","<div style=""text-align: center;"">Hull-House</d...","Jane Addams to James Henry Breasted, January 3...",[letter]
3,"Addams, Jane",1901-01-10,48,[Writing],"THE DELANO <br />\r\n<div style=""text-align: r...","Jane Addams to Richard T. Ely, January 10, 1901",[letter]
4,"Addams, Jane",1901-01-10,49,"[Books, Family]",HULL-HOUSE<br /> 335 SOUTH HALSTED STREET <br ...,"Jane Addams to Sarah Alice Addams Haldeman, Ja...",[letter]


In [12]:
len(item_df)

7741

Text cleaning helper functions 

In [13]:
# cleaning function 
# removes HTML from transcription 
# replace <br> with newline characters  
# replace others with spaces to maintain word separation
# normalizes by making all lowercase
# removes special characters and punctuation
# removes digits
def clean_text(text):
    r = '<br />'
    r1 = '<.*?>'
    r2 = '\r\n'
    r3 = '[^\w\s]' #punctuation
    r4 = '\d' #numbers
    text = re.sub(r, ' ', text)
    text = re.sub(r1, '', text)
    text = re.sub(r2, ' ', text)
    text = text.lower()
    text = re.sub(r3, '', text)
    text = re.sub(r4, '', text)
    return text 

# next step would be to split into words 
def split_words(text):
    return text.split()

# function to remove stop words from a piece of text 
# uses the stop words list from spacy 
# input must be a list of words 
def remove_stop_words(text):
    text_words_nostop = []
    for word in text:
        if word not in STOP_WORDS and word != "nbsp":
            text_words_nostop.append(word)
    return text_words_nostop

# after inspecting the text, there are lots of 1 and 2 letter words that are abbreviations 
# this is a function to remove those 
# input is a list of words 
# output is a list of words
def remove_short_words(text):
    text_no_short = []
    for word in text:
        if not(len(word) == 1 or len(word) == 2):
            text_no_short.append(word)
    return text_no_short

# there are lots of words that are like "concentrationnbsp" or "nbsp" will be between two words so it becomes one word
# this is to clean those out 
# input is a list of words
# output is a list of words 
def clean_nbsp(text):
    text_no_nbsp = []
    r = 'nbsp'
    for word in text: 
        clean_word = re.sub(r, '', word) #replace nbsp with empty string to remove 
        text_no_nbsp.append(clean_word)
    return text_no_nbsp

# most common words function 
# will return a counter object of 10 most common words 
# not necessarily part of the pipeline always, more helpful for individual pieces of text 
def most_common(text):
    return Counter(text).most_common(10)

In [14]:
#clean and replace text column 
# for text in text_list:
#     text = clean_text(text)
#     text = split_words(text)
#     text = remove_stop_words(text)
#     text = remove_short_words(text)
#     all_text.extend(text)
#first make sure all are strings 
item_df["text"] = item_df["text"].astype("str")
item_df["text"] = item_df["text"].apply(clean_text)
item_df["text"] = item_df["text"].apply(split_words)
item_df["text"] = item_df["text"].apply(remove_stop_words)
item_df["text"] = item_df["text"].apply(remove_short_words)
item_df["text"] = item_df["text"].apply(clean_nbsp)
item_df.head()

,creator,date,id,tags,text,title,type
0,"Addams, Jane",1907-09-05,17,"[Industry, Labor, Writings]","[effect, familiarity, jane, addams, hard, fami...","Effect of Familiarity, September 5, 1907",[article]
1,"Addams, Jane",1906-02-01,18,"[Education, Immigrants, Lectures]","[elective, appointive, school, board, jane, ad...","Elective or Appointive School Board, February ...",[speech]
2,"Addams, Jane",1914-01-31,19,"[Travels, Writings]","[hullhouse, south, halsted, street, chicago, j...","Jane Addams to James Henry Breasted, January 3...",[letter]
3,"Addams, Jane",1901-01-10,48,[Writing],"[delano, burlington, iowa, jany, dear, ely, at...","Jane Addams to Richard T. Ely, January 10, 1901",[letter]
4,"Addams, Jane",1901-01-10,49,"[Books, Family]","[hullhouse, south, halsted, street, chicago, j...","Jane Addams to Sarah Alice Addams Haldeman, Ja...",[letter]


Below, cleaning dates by adding a "year" to each line in the df.  

In [1]:
def date_cleaning(date): 
    x = re.search(r'^\d{4}', date)
    if not x:
        result = "NA"
    else:
        result = x[0]
    return result 

In [ ]:
item_df["year"] = item_df["date"].apply(date_cleaning)
item_df.head()

In [15]:
def bigrams(text):
    result = list(nltk.ngrams(text, 2))
    return result 

In [16]:
bigrams_list = item_df["text"].apply(bigrams)
item_df["bigrams"] = bigrams_list
item_df.head()

,creator,date,id,tags,text,title,type,bigrams
0,"Addams, Jane",1907-09-05,17,"[Industry, Labor, Writings]","[effect, familiarity, jane, addams, hard, fami...","Effect of Familiarity, September 5, 1907",[article],"[(effect, familiarity), (familiarity, jane), (..."
1,"Addams, Jane",1906-02-01,18,"[Education, Immigrants, Lectures]","[elective, appointive, school, board, jane, ad...","Elective or Appointive School Board, February ...",[speech],"[(elective, appointive), (appointive, school),..."
2,"Addams, Jane",1914-01-31,19,"[Travels, Writings]","[hullhouse, south, halsted, street, chicago, j...","Jane Addams to James Henry Breasted, January 3...",[letter],"[(hullhouse, south), (south, halsted), (halste..."
3,"Addams, Jane",1901-01-10,48,[Writing],"[delano, burlington, iowa, jany, dear, ely, at...","Jane Addams to Richard T. Ely, January 10, 1901",[letter],"[(delano, burlington), (burlington, iowa), (io..."
4,"Addams, Jane",1901-01-10,49,"[Books, Family]","[hullhouse, south, halsted, street, chicago, j...","Jane Addams to Sarah Alice Addams Haldeman, Ja...",[letter],"[(hullhouse, south), (south, halsted), (halste..."


In [68]:
flattened_bigram_list = []
for x in item_df["bigrams"]:
    flattened_bigram_list.extend(x)
bigrams_series = pd.Series(flattened_bigram_list)

In [69]:
bigrams_series.value_counts()

(jane, addams)               5378
(miss, addams)               5001
(dear, miss)                 3079
(new, york)                  2802
(miss, jane)                 2719
(hull, house)                2594
(street, chicago)            1644
(halsted, street)            1558
(south, halsted)             1492
(addams, hull)               1375
(hullhouse, south)           1323
(united, states)             1310
(house, chicago)             1194
(chicago, ill)               1141
(peace, party)               1077
(womans, peace)               976
(chicago, dear)               926
(ill, dear)                   736
(addams, signed)              671
(dear, mrs)                   579
(child, labor)                558
(chicago, illinois)           526
(york, city)                  523
(progressive, party)          507
(men, women)                  477
(years, ago)                  443
(faithfully, jane)            405
(sincerely, jane)             396
(young, people)               395
(let, know)   

In [18]:
pd.options.mode.chained_assignment = None

# Word Frequencies by Year 
<br> 
The following code generates word frequency graphs by year bar graphs for the years 1901 and 1915.  The following years are those that have enough data to generate a graph: 1900-1923. <br>  

In [ ]:
word_freq_by_year_dict = {}
for index, row in item_df.iterrows(): 
    if row["year"] not in word_freq_by_year_dict: 
        word_freq_by_year_dict[row["year"]] = row["text"]
    else:
        word_freq_by_year_dict[row["year"]].extend(row["text"])

In [19]:
word_freq_by_year_dict.keys()  

dict_keys(['1907', '1906', '1914', '1901', '1900', '1902', '1903', '1905', '1904', '1933', '1908', '1909', '1910', '1911', '1912', '1913', '1915', '1916', 'NA', '1917', '1918', '1921', '1920', '1922', '1923', '1919', '1899', '2016'])

In [23]:
x = pd.Series(word_freq_by_year_dict["1901"]).value_counts()
x.values

array([198, 192, 179, ...,   1,   1,   1])

In [94]:
 value_counts_1900 = pd.Series(word_freq_by_year_dict["1900"]).value_counts()
data1900 = [go.Bar(x=value_counts_1900.index,y=value_counts_1900.values)]
layout1900 = go.Layout(title="Most Common Words 1900",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1900 = go.Figure(data=data1900, layout=layout1900)
py.iplot(fig1900, filename='word_frequency_1900')

In [98]:
#a year - 1901 
value_counts_1901 = pd.Series(word_freq_by_year_dict["1901"]).value_counts()
data1901 = [go.Bar(x=value_counts_1901.index,y=value_counts_1901.values)]
layout1901 = go.Layout(title="Most Common Words 1901",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1901 = go.Figure(data=data1901, layout=layout1901)
py.iplot(fig1901, filename='word_frequency_1901')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~japprcnj2/0 or inside your plot.ly account where it is named 'word_frequency_1901'


In [101]:
value_counts_1902 = pd.Series(word_freq_by_year_dict["1902"]).value_counts()
data1902 = [go.Bar(x=value_counts_1902.index,y=value_counts_1902.values)]
layout1902 = go.Layout(title="Most Common Words 1902",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1902 = go.Figure(data=data1902, layout=layout1902)
py.iplot(fig1902, filename='word_frequency_1902')

In [24]:
value_counts_1903 = pd.Series(word_freq_by_year_dict["1903"]).value_counts()
data1903 = [go.Bar(x=value_counts_1903.index,y=value_counts_1903.values)]
layout1903 = go.Layout(title="Most Common Words 1903",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1903 = go.Figure(data=data1903, layout=layout1903)
py.iplot(fig1903, filename='word_frequency_1903')

In [25]:
value_counts_1904 = pd.Series(word_freq_by_year_dict["1904"]).value_counts()
data1904 = [go.Bar(x=value_counts_1904.index,y=value_counts_1904.values)]
layout1904 = go.Layout(title="Most Common Words 1904",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1904 = go.Figure(data=data1904, layout=layout1904)
py.iplot(fig1904, filename='word_frequency_1904')

In [26]:
value_counts_1905 = pd.Series(word_freq_by_year_dict["1905"]).value_counts()
data1905 = [go.Bar(x=value_counts_1905.index,y=value_counts_1905.values)]
layout1905 = go.Layout(title="Most Common Words 1905",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1905 = go.Figure(data=data1905, layout=layout1905)
py.iplot(fig1905, filename='word_frequency_1905')

In [27]:
value_counts_1906 = pd.Series(word_freq_by_year_dict["1906"]).value_counts()
data1906 = [go.Bar(x=value_counts_1906.index,y=value_counts_1906.values)]
layout1906 = go.Layout(title="Most Common Words 1906",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1906 = go.Figure(data=data1906, layout=layout1906)
py.iplot(fig1906, filename='word_frequency_1906')

In [28]:
value_counts_1907 = pd.Series(word_freq_by_year_dict["1907"]).value_counts()
data1907 = [go.Bar(x=value_counts_1907.index,y=value_counts_1907.values)]
layout1907 = go.Layout(title="Most Common Words 1907",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1907 = go.Figure(data=data1907, layout=layout1907)
py.iplot(fig1907, filename='word_frequency_1907')

In [29]:
value_counts_1908 = pd.Series(word_freq_by_year_dict["1908"]).value_counts()
data1908 = [go.Bar(x=value_counts_1908.index,y=value_counts_1908.values)]
layout1908 = go.Layout(title="Most Common Words 1908",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1908 = go.Figure(data=data1908, layout=layout1908)
py.iplot(fig1908, filename='word_frequency_1908')

In [30]:
value_counts_1909 = pd.Series(word_freq_by_year_dict["1909"]).value_counts()
data1909 = [go.Bar(x=value_counts_1909.index,y=value_counts_1909.values)]
layout1909 = go.Layout(title="Most Common Words 1909",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1909 = go.Figure(data=data1909, layout=layout1909)
py.iplot(fig1909, filename='word_frequency_1909')

In [31]:
value_counts_1910 = pd.Series(word_freq_by_year_dict["1910"]).value_counts()
data1910 = [go.Bar(x=value_counts_1910.index,y=value_counts_1910.values)]
layout1910 = go.Layout(title="Most Common Words 1910",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1910 = go.Figure(data=data1910, layout=layout1910)
py.iplot(fig1910, filename='word_frequency_1910')

In [32]:
value_counts_1911 = pd.Series(word_freq_by_year_dict["1911"]).value_counts()
data1911 = [go.Bar(x=value_counts_1911.index,y=value_counts_1911.values)]
layout1911 = go.Layout(title="Most Common Words 1911",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1911 = go.Figure(data=data1911, layout=layout1911)
py.iplot(fig1911, filename='word_frequency_1911')

In [33]:
value_counts_1912 = pd.Series(word_freq_by_year_dict["1912"]).value_counts()
data1912 = [go.Bar(x=value_counts_1912.index,y=value_counts_1912.values)]
layout1912 = go.Layout(title="Most Common Words 1912",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1912 = go.Figure(data=data1912, layout=layout1912)
py.iplot(fig1912, filename='word_frequency_1912')

In [34]:
value_counts_1913 = pd.Series(word_freq_by_year_dict["1913"]).value_counts()
data1913 = [go.Bar(x=value_counts_1913.index,y=value_counts_1913.values)]
layout1913 = go.Layout(title="Most Common Words 1913",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1913 = go.Figure(data=data1913, layout=layout1913)
py.iplot(fig1913, filename='word_frequency_1913')

In [35]:
value_counts_1914 = pd.Series(word_freq_by_year_dict["1914"]).value_counts()
data1914 = [go.Bar(x=value_counts_1914.index,y=value_counts_1914.values)]
layout1914 = go.Layout(title="Most Common Words 1914",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1914 = go.Figure(data=data1914, layout=layout1914)
py.iplot(fig1914, filename='word_frequency_1914')

In [36]:
value_counts_1915 = pd.Series(word_freq_by_year_dict["1915"]).value_counts()
data1915 = [go.Bar(x=value_counts_1915.index,y=value_counts_1915.values)]
layout1915 = go.Layout(title="Most Common Words 1915",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1915 = go.Figure(data=data1915, layout=layout1915)
py.iplot(fig1915, filename='word_frequency_1915')

In [37]:
value_counts_1916 = pd.Series(word_freq_by_year_dict["1916"]).value_counts()
data1916 = [go.Bar(x=value_counts_1916.index,y=value_counts_1916.values)]
layout1916 = go.Layout(title="Most Common Words 1916",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1916 = go.Figure(data=data1916, layout=layout1916)
py.iplot(fig1916, filename='word_frequency_1916')

In [38]:
value_counts_1917 = pd.Series(word_freq_by_year_dict["1917"]).value_counts()
data1917 = [go.Bar(x=value_counts_1917.index,y=value_counts_1917.values)]
layout1917 = go.Layout(title="Most Common Words 1917",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1917 = go.Figure(data=data1917, layout=layout1917)
py.iplot(fig1917, filename='word_frequency_1917')

In [39]:
value_counts_1918 = pd.Series(word_freq_by_year_dict["1918"]).value_counts()
data1918 = [go.Bar(x=value_counts_1918.index,y=value_counts_1918.values)]
layout1918 = go.Layout(title="Most Common Words 1918",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1918 = go.Figure(data=data1918, layout=layout1918)
py.iplot(fig1918, filename='word_frequency_1918')

In [40]:
value_counts_1919 = pd.Series(word_freq_by_year_dict["1919"]).value_counts()
data1919 = [go.Bar(x=value_counts_1919.index,y=value_counts_1919.values)]
layout1919 = go.Layout(title="Most Common Words 1919",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1919 = go.Figure(data=data1919, layout=layout1919)
py.iplot(fig1919, filename='word_frequency_1919')

In [41]:
value_counts_1920 = pd.Series(word_freq_by_year_dict["1920"]).value_counts()
data1920 = [go.Bar(x=value_counts_1920.index,y=value_counts_1920.values)]
layout1920 = go.Layout(title="Most Common Words 1920",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1920 = go.Figure(data=data1920, layout=layout1920)
py.iplot(fig1920, filename='word_frequency_1920')

In [42]:
value_counts_1921 = pd.Series(word_freq_by_year_dict["1921"]).value_counts()
data1921 = [go.Bar(x=value_counts_1921.index,y=value_counts_1921.values)]
layout1921 = go.Layout(title="Most Common Words 1921",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1921 = go.Figure(data=data1921, layout=layout1921)
py.iplot(fig1921, filename='word_frequency_1921')

In [43]:
value_counts_1922 = pd.Series(word_freq_by_year_dict["1922"]).value_counts()
data1922 = [go.Bar(x=value_counts_1922.index,y=value_counts_1922.values)]
layout1922 = go.Layout(title="Most Common Words 1922",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1922 = go.Figure(data=data1922, layout=layout1922)
py.iplot(fig1922, filename='word_frequency_1922')

In [44]:
value_counts_1923 = pd.Series(word_freq_by_year_dict["1923"]).value_counts()
data1923 = [go.Bar(x=value_counts_1923.index,y=value_counts_1923.values)]
layout1923 = go.Layout(title="Most Common Words 1923",
                xaxis=dict(title='Words'),
                yaxis=dict(title='Number of Appearances in Text'))
fig1923 = go.Figure(data=data1923, layout=layout1923)
py.iplot(fig1923, filename='word_frequency_1923')

In [92]:
import plotly.tools as tls

tls.get_embed('https://plot.ly/~japprcnj2/2')

'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~japprcnj2/2.embed" height="525" width="100%"></iframe>'

In [21]:
# value counts series for each year 
value_counts_1900 = pd.Series(word_freq_by_year_dict["1900"]).value_counts()
value_counts_1901 = pd.Series(word_freq_by_year_dict["1901"]).value_counts()
value_counts_1902 = pd.Series(word_freq_by_year_dict["1902"]).value_counts()
value_counts_1903 = pd.Series(word_freq_by_year_dict["1903"]).value_counts()
value_counts_1904 = pd.Series(word_freq_by_year_dict["1904"]).value_counts()
value_counts_1905 = pd.Series(word_freq_by_year_dict["1905"]).value_counts()
value_counts_1906 = pd.Series(word_freq_by_year_dict["1906"]).value_counts()
value_counts_1907 = pd.Series(word_freq_by_year_dict["1907"]).value_counts()
value_counts_1908 = pd.Series(word_freq_by_year_dict["1908"]).value_counts()
value_counts_1909 = pd.Series(word_freq_by_year_dict["1909"]).value_counts()
value_counts_1910 = pd.Series(word_freq_by_year_dict["1910"]).value_counts()
value_counts_1911 = pd.Series(word_freq_by_year_dict["1911"]).value_counts()
value_counts_1912 = pd.Series(word_freq_by_year_dict["1912"]).value_counts()
value_counts_1913 = pd.Series(word_freq_by_year_dict["1913"]).value_counts()
value_counts_1914 = pd.Series(word_freq_by_year_dict["1914"]).value_counts()
value_counts_1915 = pd.Series(word_freq_by_year_dict["1915"]).value_counts()
value_counts_1916 = pd.Series(word_freq_by_year_dict["1916"]).value_counts()
value_counts_1917 = pd.Series(word_freq_by_year_dict["1917"]).value_counts()
value_counts_1918 = pd.Series(word_freq_by_year_dict["1918"]).value_counts()
value_counts_1919 = pd.Series(word_freq_by_year_dict["1919"]).value_counts()
value_counts_1920 = pd.Series(word_freq_by_year_dict["1920"]).value_counts()
value_counts_1921 = pd.Series(word_freq_by_year_dict["1921"]).value_counts()
value_counts_1922 = pd.Series(word_freq_by_year_dict["1922"]).value_counts()
value_counts_1923 = pd.Series(word_freq_by_year_dict["1923"]).value_counts()

In [22]:
value_counts_series_list = []
value_counts_series_list.append(value_counts_1900)
value_counts_series_list.append(value_counts_1901)
value_counts_series_list.append(value_counts_1902)
value_counts_series_list.append(value_counts_1903)
value_counts_series_list.append(value_counts_1904)
value_counts_series_list.append(value_counts_1905)
value_counts_series_list.append(value_counts_1906)
value_counts_series_list.append(value_counts_1907)
value_counts_series_list.append(value_counts_1908)
value_counts_series_list.append(value_counts_1909)
value_counts_series_list.append(value_counts_1910)
value_counts_series_list.append(value_counts_1911)
value_counts_series_list.append(value_counts_1912)
value_counts_series_list.append(value_counts_1913)
value_counts_series_list.append(value_counts_1914)
value_counts_series_list.append(value_counts_1915)
value_counts_series_list.append(value_counts_1916)
value_counts_series_list.append(value_counts_1917)
value_counts_series_list.append(value_counts_1918)
value_counts_series_list.append(value_counts_1919)
value_counts_series_list.append(value_counts_1920)
value_counts_series_list.append(value_counts_1921)
value_counts_series_list.append(value_counts_1922)
value_counts_series_list.append(value_counts_1923)

In [37]:
def generate_word_freq_over_time(word):
    word_freq_over_time = {} #dict where key:value is year:count
    year = 1900
    for vc_list in value_counts_series_list:
        try: 
            word_freq_over_time[year] = vc_list[word] 
        except: 
            word_freq_over_time[year] = 0
        year = year + 1
    return word_freq_over_time

In [38]:
def generate_rel_freq_over_time(word):
    word_freq_over_time = {} #dict where key:value is year:rel freq 
    year = 1900
    for vc_list in value_counts_series_list:
        try: 
            word_freq_over_time[year] = vc_list[word] / len(word_freq_by_year_dict[year])
        except: 
            word_freq_over_time[year] = 0
        year = year + 1
    return word_freq_over_time

In [24]:
flattened_text_list = []
for x in item_df["text"]:
    flattened_text_list.extend(x)

In [25]:
#for testing 
one_thousand_words = pd.Series(flattened_text_list).value_counts()[:1000]
ten_words = pd.Series(flattened_text_list).value_counts()[:10]

In [26]:
word = ten_words.keys()[0]
result = generate_word_freq_over_time(word)
result = pd.Series(result)

In [48]:
data = []
for word in ten_words.keys()[:3]:
    result = generate_word_freq_over_time(word)
    trace = go.Scatter(
        x = list(result.keys()),
        y = list(result.values()),
        name = word
    )
    data.append(trace)

In [50]:
updatemenus = list([
    dict(active=-1,
         buttons=list([   
            dict(label = data[0]["name"],
                 method = 'update',
                 args = [{'visible': [True, False, False]},
                         {'title': data[0]["name"]}]),
            dict(label = data[1]["name"],
                 method = 'update',
                 args = [{'visible': [False, True, False]},
                         {'title': data[1]["name"]}]),
            dict(label = data[2]["name"],
                 method = 'update',
                 args = [{'visible': [False, False, True]},
                         {'title': data[2]["name"]}])
        ])
    )
])

In [51]:
layout = go.Layout(title=word,
            xaxis=dict(title='Year'),
            yaxis=dict(title='Frequency of Use'), updatemenus=updatemenus)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="word")

In [76]:
item_df.head()

,creator,date,id,tags,text,title,type,bigrams,year
0,"Addams, Jane",1907-09-05,17,"[Industry, Labor, Writings]","[effect, familiarity, jane, addams, hard, fami...","Effect of Familiarity, September 5, 1907",[article],"[(effect, familiarity), (familiarity, jane), (...",1907
1,"Addams, Jane",1906-02-01,18,"[Education, Immigrants, Lectures]","[elective, appointive, school, board, jane, ad...","Elective or Appointive School Board, February ...",[speech],"[(elective, appointive), (appointive, school),...",1906
2,"Addams, Jane",1914-01-31,19,"[Travels, Writings]","[hullhouse, south, halsted, street, chicago, j...","Jane Addams to James Henry Breasted, January 3...",[letter],"[(hullhouse, south), (south, halsted), (halste...",1914
3,"Addams, Jane",1901-01-10,48,[Writing],"[delano, burlington, iowa, jany, dear, ely, at...","Jane Addams to Richard T. Ely, January 10, 1901",[letter],"[(delano, burlington), (burlington, iowa), (io...",1901
4,"Addams, Jane",1901-01-10,49,"[Books, Family]","[hullhouse, south, halsted, street, chicago, j...","Jane Addams to Sarah Alice Addams Haldeman, Ja...",[letter],"[(hullhouse, south), (south, halsted), (halste...",1901


In [78]:
jane_item_df = item_df[item_df["creator"] == "Addams, Jane"]
len(jane_item_df)

3189

In [81]:
item_df["creator"].value_counts()

Addams, Jane                                              3189
Kellogg, Paul Underwood                                    108
Karsten, Eleanor Daggett                                   100
Addams, Jane                                                98
Macmillan Company                                           70
Post, Alice Thacher                                         45
Schwimmer, Rosika                                           42
Thomas, Harriet Park                                        41
Ely, Richard T.                                             39
Lochner, Louis Paul                                         37
Balch, Emily Greene                                         36
Manus, Rosa                                                 32
Hyers, Mabel L.                                             31
Mead, Lucia Ames                                            31
Lindsey, Ben B.                                             29
Breckinridge, Sophonisba P.                            

In [80]:
Kellogg, Paul Underwood

1

## sentiment analysis

can talk about how we weren't really getting any meaningful results - nothing interesting enough on a large scale that added valuable information 

In [25]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/catieo/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [26]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sent_analyzer = SentimentIntensityAnalyzer()
def sentiment_scores(docx):
    return sent_analyzer.polarity_scores(docx.text)

/anaconda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning:

The twython library has not been installed. Some functionality from the twitter package will not be available.



In [27]:
nlp = spacy.load('en')
from spacy.tokens import Doc
Doc.set_extension("sentimenter",getter=sentiment_scores)

In [48]:
sep = " "
test_sent = sep.join(item_df["text"][1])

In [50]:
nlp(test_sent)._.sentimenter

{'compound': 1.0, 'neg': 0.096, 'neu': 0.691, 'pos': 0.213}

I think it's -1 to 1 but that doesn't really make sense here? also takes a really long time to run. It looks like compound is also 1.0, which isn't really helpful, but the other values are fluctuating so maybe could do something with that

or we could try this guy: 

In [ ]:
# returns "positive", "negative" or "neutral" 
def sentiment_analysis(text):
  base_url = "http://api.datumbox.com/1.0/SentimentAnalysis.json"
  url_params = { "api_key" : '6e08fe05ac351af94c7d4b625385e703', "text" : text }
  response = requests.get(base_url, url_params)
  result = json.loads(response.text)
  return result["output"]["result"]